In [1]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [2]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def route(text):
    if len(text["text"]) < 10:
        return RunnableLambda(lambda x: "small text input")
    else:
        return RunnableLambda(lambda x: "big text input")
    
full_chain = RunnablePassthrough() | RunnableLambda(route)

In [42]:
info = {
    "text": "a",
    "option": "z",
}

In [12]:
def lul(info):
    return RunnableLambda(lambda x: x["text"] + "lul")

lul_chain = {
    "info": RunnablePassthrough(),
    "output": RunnableLambda(lul)
} | RunnablePassthrough()
lul_chain.invoke(info)

{'info': {'text': 'a', 'option': 'z'}, 'output': 'alul'}

In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
import os
from langchain_openai import ChatOpenAI
import openai
from abrege.extractive_summary import EmbeddingModel
from abrege.summary_chain import new_summarize_chain

key = os.environ["OPENAI_API_KEY"]
base = os.environ["OPENAI_API_BASE"]
e_key = os.environ["OPENAI_EMBEDDING_API_KEY"]
e_base = os.environ["OPENAI_EMBEDDING_API_BASE"]

client = openai.OpenAI(
    base_url=e_base,
    api_key=e_key
)
em = EmbeddingModel(client)

llm = ChatOpenAI(
    base_url=base,
    api_key=key,
)

chain = new_summarize_chain()

chain.invoke({
    "text": "J'adore le chocolat",
    "llm": llm,
    "embedding_model": em,
    "method": "map_reduce",
    "size": 100
})

"Les thèmes principaux de ces documents se rassemblent autour du amour du chocolat, comme le montre J'adore le chocolat (J'adore le chocolat). Les documents mettent en évidence les avantages de la consommation du chocolat et son rôle dans la promotion d'un mode de vie sain. Ils soulignent également l'importance des ingrédients de qualité et des méthodes de production durables pour garantir que les consommateurs peuvent apprécier le chocolat tout en soutenant des pratiques éthiques et environnementales."

In [5]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
chain = RunnablePassthrough.assign(test=lambda x: "haha")
larger_chain = RunnableParallel(info=chain, extras=RunnableLambda(lambda x: "extras"))
larger_chain.invoke({"input": "bb"})

{'info': {'input': 'bb', 'test': 'haha'}, 'extras': 'extras'}

In [32]:
from abrege.summary_chain import refine_chain, input_chain


chain = input_chain | refine_chain

chain.invoke({
    "text": "J'adore le chocolat",
    "llm": llm,
    "embedding_model": em,
    "method": "refine",
    "size": 100
})

{'summary_english': '"J\'adore le chocolat" is a French phrase that translates to "I love chocolate." It expresses a strong fondness or appreciation for chocolate as a sweet treat. The phrase can be used in various contexts, such'}

In [45]:
from abrege.summary_chain import summarize_chain_builder

chain = summarize_chain_builder(llm=llm, embedding_model=em)
chain.invoke({"text": "J'aime le chocolat"})

{'text': "J'aime le chocolat", 'llm': None, 'embedding_model': None, 'context_size': None, 'language': None, 'method': None, 'size': None, 'summarize_template': None, 'map_template': None, 'reduce_template': None, 'question_template': None, 'refine_template': None}
{'text': "J'aime le chocolat", 'llm': None, 'embedding_model': None, 'context_size': 2500, 'language': 'french', 'method': None, 'size': 200, 'summarize_template': '\nThe following is a collection of extract from a text (or the entire text itself)\n{text}\nTake these and distill it into a consolidated summary in at most {size} words\n', 'map_template': 'The following is a set of documents\n{docs}\nBased on this list of docs, please identify the main themes\nHelpful Answer:', 'reduce_template': 'The following is set of summaries\n{docs}\nTake these and distill it into a final, consolidated summary written of the main themes in at most {size} words.\nHelpful Answer:', 'question_template': 'Write a concise summary of the follow

ValueError: 

In [44]:
for k, v in info.items():
    print(k, v)

Traceback (most recent call last):
  File "/root/.local/lib/python3.11/site-packages/IPython/core/completer.py", line 3171, in _complete
    result = matcher(context)
             ^^^^^^^^^^^^^^^^
  File "/root/.local/lib/python3.11/site-packages/IPython/core/completer.py", line 2099, in magic_matcher
    matches = self.magic_matches(text)
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/lib/python3.11/site-packages/IPython/core/completer.py", line 2132, in magic_matches
    global_matches = self.global_matches(bare_text)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/lib/python3.11/site-packages/IPython/core/completer.py", line 1113, in global_matches
    for word in lst:
RuntimeError: dictionary changed size during iteration


text a
option z
